### define

In [1]:
from resources.utils import *
from gensim.models import Word2Vec

mc['github'].list_collection_names()

['users_followers_embeddings',
 'followers',
 'users_following_embeddings',
 'followers_all',
 'users_watches',
 'repos_watches_embeddings',
 'users',
 'projects',
 'repos_stars_embeddings',
 'following']

## users

In [8]:
model_name = 'users_followers_emb_sg_d300_w30_mc3_e5'
model = Word2Vec.load('./models/' + model_name + '.bin')
print(model)

Word2Vec(vocab=1446832, size=300, alpha=0.025)


In [9]:
good_uids = []

for uid, r in model.wv.vocab.items():
    if r.count >= 100:
        good_uids.append(uid)
len(good_uids)

24435

In [12]:
# username = 'durov'
# username = 'ping'
# username = 'codebox'
username = 'JakeWharton'
# username = 'kennethreitz'

In [13]:
uid = mc['github']['users'].find_one({'login': username}, {})['_id']
uid = str(uid)
if uid in model.wv.vocab:
    print(f'stored, {model.wv.vocab[uid].count} word count')
else:
    print('not stored')
uid

stored, 24 word count


'896'

In [14]:
df = pd.DataFrame(model.wv.similar_by_word(uid, topn=10), columns=['uid', 'similarity']).set_index('uid')
df['login'] = df.index.map(lambda x: mc['github']['users'].find_one(int(x), ['login'])['login'])
df['words_count'] = df.index.map(lambda x: model.wv.vocab[x].count)
df

,similarity,login,words_count
uid,,,
8220,0.983574,rashiqahmad,7
1955,0.983191,thiagolocatelli,39
8240,0.982011,jacintosan,6
276,0.979131,soe,5
5830,0.979046,doyonghoon,21
8221,0.978960,cyrilmottier,8
8122,0.978828,iDG,4
2611,0.978275,arriolac,67
6809,0.976906,jferlisi,14


In [15]:
df['login'].map(lambda x: f'https://github.com/{x}').map(print);

https://github.com/rashiqahmad
https://github.com/thiagolocatelli
https://github.com/jacintosan
https://github.com/soe
https://github.com/doyonghoon
https://github.com/cyrilmottier
https://github.com/iDG
https://github.com/arriolac
https://github.com/jferlisi
https://github.com/pkafka


## repos

In [2]:
repo_model_name = 'repos_watches_emb_sg_d300_w30_mc3_e5'
model = Word2Vec.load('./models/' + repo_model_name + '.bin')
print(model)

Word2Vec(vocab=2371786, size=300, alpha=0.025)


In [3]:
good_rids = []

for rid, r in model.wv.vocab.items():
    if r.count >= 1000:
        good_rids.append(rid)
len(good_rids)

18831

In [4]:
# repo_name = 'rg3/youtube-dl'
# repo_name = 'python/cpython'
# repo_name = 'matplotlib/basemap'
# repo_name = 'networkx/networkx'
# repo_name = 'altair-viz/altair'
# repo_name = 'dmlc/xgboost'
repo_name = 'TeamHG-Memex/eli5'

In [5]:
r = mc['github']['projects'].find_one({'url': repo_name}, {})
rid = str(r['_id'])
if rid in model.wv.vocab:
    print(f'stored, {model.wv.vocab[rid].count} word count')
else:
    print('not stored')
rid

stored, 1229 word count


'45799319'

In [6]:
df = pd.DataFrame(model.wv.similar_by_word(rid, topn=10), columns=['rid', 'similarity']).set_index('rid')
df['url'] = df.index.map(lambda x: mc['github']['projects'].find_one(int(x), ['url'])['url'])
df['words_count'] = df.index.map(lambda x: model.wv.vocab[x].count)
df

/opt/conda/lib/python3.7/site-packages/gensim/matutils.py:737: FutureWarning:

Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.



,similarity,url,words_count
rid,,,
41436405,0.804646,scikit-learn-contrib/imbalanced-learn,2890
44469343,0.801859,anttttti/Wordbatch,376
44573709,0.787377,scikit-learn-contrib/hdbscan,768
51973607,0.781602,slundberg/shap,3839
42893532,0.781349,limexp/xgbfir,273
46287654,0.780820,interpretable-ml/iml,43
45057062,0.778374,bobye/libffm-regression,10
44728585,0.772377,madrugado/gia-corpus,4
45805290,0.770090,commonsense/conceptnet-numberbatch,658


In [7]:
df['url'].map(lambda x: f'https://github.com/{x}').map(print);

https://github.com/scikit-learn-contrib/imbalanced-learn
https://github.com/anttttti/Wordbatch
https://github.com/scikit-learn-contrib/hdbscan
https://github.com/slundberg/shap
https://github.com/limexp/xgbfir
https://github.com/interpretable-ml/iml
https://github.com/bobye/libffm-regression
https://github.com/madrugado/gia-corpus
https://github.com/commonsense/conceptnet-numberbatch
https://github.com/conda-forge/tpot-feedstock


## sandbox

In [45]:
cur = mc['github']['followers'].find({}, ['followers_count']).sort([("followers_count", -1)]).limit(10)

s = pd.DataFrame(list(cur)).set_index('_id')['followers_count']
s

_id
5203      95258
896       53972
616741    48027
376498    41660
18556     39688
417948    36377
6240      36128
3871      31648
1779      31553
796       27757
Name: followers_count, dtype: int64

In [48]:
top_foll = s.index.to_list()
top_foll

[5203, 896, 616741, 376498, 18556, 417948, 6240, 3871, 1779, 796]

In [50]:
cur = mc['github']['users'].find({"_id": {"$in": top_foll}})
df = pd.DataFrame(list(cur)).set_index('_id')
df

,city,company,country_code,created_at,deleted,fake,lat,location,login,long,state,type
_id,,,,,,,,,,,,
796,None,@digitalocean,None,2009-08-26 19:17:47,False,False,0.000000,"Winchester, VA, USA",kennethreitz,0.000000,None,USR
896,Pittsburgh,"Google, Inc.",us,2009-03-24 15:09:53,False,False,40.440625,"Pittsburgh, PA, USA",JakeWharton,-79.995886,Allegheny County,USR
1779,Palo Alto,"Google Chrome, ♥z",us,2008-12-08 23:39:23,False,False,37.441883,Palo Alto,paulirish,-122.143019,Santa Clara County,USR
3871,None,@avajs @xojs @chalk,None,2009-12-20 21:57:02,False,False,0.000000,° ☾ ☆ ¸. ¸ ★ :. . • ○ ° ★,sindresorhus,0.000000,None,USR
5203,Portland,Linux Foundation,us,2011-09-03 13:26:22,False,False,45.512231,"Portland, OR",torvalds,-122.658719,Multnomah County,USR
6240,Mountain View,Google,us,2009-08-01 16:39:25,False,False,37.386052,"Mountain View, California",addyosmani,-122.083851,Santa Clara County,USR
18556,None,None,None,2010-11-28 00:05:40,False,False,0.000000,New Jersey / China,yyx990803,0.000000,None,USR
376498,Victoria,Apex,ca,2008-11-09 04:33:10,False,False,48.428421,"Victoria, BC, Canada",Tj,-123.365644,Capital,USR
417948,London,@facebook,gb,2011-05-25 16:18:31,False,False,51.507351,"London, UK",gaearon,-0.127758,Greater London,USR


In [55]:
df['login'].map(lambda x: f'https://github.com/{x}').map(print);

https://github.com/kennethreitz
https://github.com/JakeWharton
https://github.com/paulirish
https://github.com/sindresorhus
https://github.com/torvalds
https://github.com/addyosmani
https://github.com/yyx990803
https://github.com/Tj
https://github.com/gaearon
https://github.com/ruanyf
